In [1]:
import pandas as pd

df = pd.read_csv("Afluencia_Metro/afluenciastc_simple_09_2025.csv")


import sys
import numpy as np

# Opción 1: Forzar impresión en consola (Desactiva el truncamiento)
np.set_printoptions(threshold=sys.maxsize)

# Se recomienda ordenar para agrupar errores tipográficos similares
estaciones_unicas = sorted(df["estacion"].astype(str).unique())

print("--- LISTADO COMPLETO DE ESTACIONES ---")
print(np.array(estaciones_unicas))

# Opción 2: Exportar a .txt (Recomendado para listas masivas como colonias CDMX)
# Esto permite buscar con CTRL+F y evita saturar la terminal
ruta_txt = r"Data_Tratado\Estaciones_Unicas.txt"
with open(ruta_txt, "w", encoding="utf-8") as f:
    for estacion in estaciones_unicas:
        f.write(f"{estacion}\n")

print(f"\nSe generó archivo de revisión en: {ruta_txt}")

--- LISTADO COMPLETO DE ESTACIONES ---
['Acatitla' 'Aculco' 'AgrÃ\xadcola Oriental' 'Allende' 'Apatlaco'
 'Aquiles SerdÃ¡n' 'AragÃ³n' 'Atlalilco' 'Auditorio' 'Autobuses del Norte'
 'Balbuena' 'Balderas' 'Barranca del Muerto' 'Bellas Artes' 'Bondojito'
 'Bosque de AragÃ³n' 'Boulevard Puerto AÃ©reo' 'Buenavista' 'Calle 11'
 'Camarones' 'Canal de San Juan' 'Canal del Norte' 'Candelaria'
 'Centro MÃ©dico' 'Cerro de la Estrella' 'Chabacano' 'Chapultepec'
 'Chilpancingo' 'Ciudad Azteca' 'Ciudad Deportiva' 'Colegio Militar'
 'ConstituciÃ³n de 1917' 'Constituyentes' 'Consulado' 'Copilco'
 'CoyoacÃ¡n' 'Coyuya' 'Cuatro Caminos' 'CuauhtÃ©moc' 'CuitlÃ¡huac'
 'CulhuacÃ¡n' 'Deportivo 18 de Marzo' 'Deportivo OceanÃ\xada'
 'DivisiÃ³n del Norte' 'Doctores' 'Ecatepec' 'Eduardo Molina'
 'Eje Central' 'El Rosario' 'Ermita' 'EscuadrÃ³n 201'
 'EtiopÃ\xada/Plaza de la Transparencia' 'Eugenia'
 'FerrerÃ\xada/Arena Ciudad de MÃ©xico' 'Fray Servando'
 'Garibaldi/Lagunilla' 'General Anaya' 'Guelatao' 'Guerrero'


In [2]:
import os

# Cargar dataset
df = pd.read_csv("Afluencia_Metro/afluenciastc_simple_09_2025.csv")

# 1. Función de Reparación de Texto (Mojibake)
def reparar_nombre(texto):
    if pd.isna(texto):
        return "DESCONOCIDO"
    try:
        # Intenta revertir la codificación incorrecta (Ã³ -> ó)
        # Se prueba latin-1 primero, si falla, se devuelve el original
        return texto.encode('latin-1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return texto

# 2. Función de Normalización (Mayúsculas sin acentos)
mapa_acentos = str.maketrans("ÁÉÍÓÚÑÄËÏÖÜ", "AEIOUNAEIOU")

def normalizar_estacion(texto):
    # 1. Reparar codificación
    texto_reparado = reparar_nombre(texto)
    # 2. Convertir a string, mayúsculas y quitar espacios extra
    texto_limpio = str(texto_reparado).strip().upper()
    # 3. Eliminar acentos
    return texto_limpio.translate(mapa_acentos)

# Aplicar transformación
print("Procesando nombres de estaciones...")
df['ESTACION_NORMALIZADA'] = df['estacion'].apply(normalizar_estacion)

# 3. Correcciones manuales específicas (opcional pero recomendado)
# Unificar nombres compuestos que cambian por año (ej. cambio de nombre Zócalo)
correcciones = {
    'ZOCALO/TENOCHTITLAN': 'ZOCALO',
    'ETIOPIA/PLAZA DE LA TRANSPARENCIA': 'ETIOPIA',
    'VIVEROS/DERECHOS HUMANOS': 'VIVEROS',
    'FERRERIA/ARENA CIUDAD DE MEXICO': 'FERRERIA',
    'LINDAVISTA': 'LINDAVISTA',  # A veces aparece como Lindavista/Cine...
    'LA VILLA/BASILICA': 'LA VILLA'
}
# Nota: Si prefieres mantener el nombre largo, omite el reemplazo
df['ESTACION_NORMALIZADA'] = df['ESTACION_NORMALIZADA'].replace(correcciones)

# 4. Validación
estaciones_limpias = sorted(df['ESTACION_NORMALIZADA'].unique())
print(f"\nTotal estaciones únicas detectadas: {len(estaciones_limpias)}")
print("Muestra (primeros 10):")
print(np.array(estaciones_limpias[:10]))

Procesando nombres de estaciones...

Total estaciones únicas detectadas: 163
Muestra (primeros 10):
['ACATITLA' 'ACULCO' 'AGRICOLA ORIENTAL' 'ALLENDE' 'APATLACO'
 'AQUILES SERDAN' 'ARAGON' 'ATLALILCO' 'AUDITORIO' 'AUTOBUSES DEL NORTE']


In [3]:
display(df.head())

,fecha,anio,mes,linea,estacion,afluencia,ESTACION_NORMALIZADA
0,2010-01-01,2010,Enero,Linea 1,Zaragoza,20227,ZARAGOZA
1,2010-01-01,2010,Enero,Linea 1,Isabel la CatÃ³lica,6487,ISABEL LA CATOLICA
2,2010-01-01,2010,Enero,Linea 1,Moctezuma,10304,MOCTEZUMA
3,2010-01-01,2010,Enero,Linea 1,Pino SuÃ¡rez,8679,PINO SUAREZ
4,2010-01-01,2010,Enero,Linea 1,GÃ³mez FarÃ­as,19499,GOMEZ FARIAS


In [4]:
df["ESTACION_NORMALIZADA"].unique()

array(['ZARAGOZA', 'ISABEL LA CATOLICA', 'MOCTEZUMA', 'PINO SUAREZ',
       'GOMEZ FARIAS', 'DEPORTIVO 18 DE MARZO', 'LA VILLA', 'PANTITLAN',
       'ACULCO', 'VELODROMO', 'AUTOBUSES DEL NORTE', 'MISTERIOS',
       'CONSTITUYENTES', 'REFINERIA', 'ETIOPIA', 'POLANCO',
       'CANAL DEL NORTE', 'BONDOJITO', 'SANTA ANITA', 'POPOTLA',
       'GENERAL ANAYA', 'DIVISION DEL NORTE', 'ZAPATA', 'FRAY SERVANDO',
       'HANGARES', 'EL ROSARIO', 'FERRERIA', 'INSTITUTO DEL PETROLEO',
       'COYUYA', 'APATLACO', 'LA VIGA', 'CHABACANO', 'SAN JUAN DE LETRAN',
       'BARRANCA DEL MUERTO', 'GARIBALDI/LAGUNILLA', 'BUENAVISTA',
       'GUERRERO', 'OCEANIA', 'NEZAHUALCOYOTL', 'REVOLUCION',
       'INDIOS VERDES', 'HIDALGO', 'COLEGIO MILITAR', 'NORMAL',
       'SAN COSME', 'JUAREZ', 'OLIMPICA', 'CIUDAD AZTECA', 'TLALTENCO',
       'INSURGENTES SUR', 'CULHUACAN', 'CALLE 11', 'CHILPANCINGO',
       'PENON VIEJO', 'CANAL DE SAN JUAN', 'LOMAS ESTRELLA',
       'SAN ANDRES TOMATLAN', 'ERMITA', 'XOLA', 'SALTO 

In [5]:
# 1. Diccionario Maestro de Coordenadas (Cobertura Total: 163 Estaciones)
# Fuente: Datos Geoestadísticos CDMX y STC Metro
coords_metro = {
    'ACATITLA': (19.3644, -99.0058),
    'ACULCO': (19.3795, -99.1084),
    'AGRICOLA ORIENTAL': (19.4116, -99.0747),
    'ALLENDE': (19.4354, -99.1374),
    'APATLACO': (19.3783, -99.1165),
    'AQUILES SERDAN': (19.4896, -99.1947),
    'ARAGON': (19.4513, -99.0967),
    'ATLALILCO': (19.3556, -99.1035),
    'AUDITORIO': (19.4243, -99.1925),
    'AUTOBUSES DEL NORTE': (19.4792, -99.1432),
    'BALBUENA': (19.4230, -99.1027),
    'BALDERAS': (19.4273, -99.1491),
    'BARRANCA DEL MUERTO': (19.3615, -99.1893),
    'BELLAS ARTES': (19.4363, -99.1418),
    'BONDOJITO': (19.4628, -99.1171),
    'BOSQUE DE ARAGON': (19.4580, -99.0699),
    'BOULEVARD PUERTO AEREO': (19.4195, -99.0960),
    'BUENAVISTA': (19.4463, -99.1534),
    'CALLE 11': (19.3308, -99.0963),
    'CAMARONES': (19.4789, -99.1866),
    'CANAL DE SAN JUAN': (19.3983, -99.0573),
    'CANAL DEL NORTE': (19.4485, -99.1163),
    'CANDELARIA': (19.4290, -99.1197),
    'CENTRO MEDICO': (19.4067, -99.1552),
    'CERRO DE LA ESTRELLA': (19.3556, -99.0863),
    'CHABACANO': (19.4085, -99.1358),
    'CHAPULTEPEC': (19.4206, -99.1763),
    'CHILPANCINGO': (19.4060, -99.1685),
    'CIUDAD AZTECA': (19.5346, -99.0275),
    'CIUDAD DEPORTIVA': (19.4081, -99.0911),
    'COLEGIO MILITAR': (19.4481, -99.1728),
    'CONSTITUCION DE 1917': (19.3465, -99.0754),
    'CONSTITUYENTES': (19.4121, -99.1920),
    'CONSULADO': (19.4583, -99.1135),
    'COPILCO': (19.3359, -99.1770),
    'COYOACAN': (19.3620, -99.1697),
    'COYUYA': (19.4005, -99.1102),
    'CUATRO CAMINOS': (19.4599, -99.2158),
    'CUAUHTEMOC': (19.4258, -99.1549),
    'CUITLAHUAC': (19.4678, -99.1722),
    'CULHUACAN': (19.3458, -99.1245),
    'DEPORTIVO 18 DE MARZO': (19.4842, -99.1251),
    'DEPORTIVO OCEANIA': (19.4510, -99.0772),
    'DIVISION DEL NORTE': (19.3797, -99.1593),
    'DOCTORES': (19.4216, -99.1436),
    'ECATEPEC': (19.5138, -99.0370),
    'EDUARDO MOLINA': (19.4503, -99.1026),
    'EJE CENTRAL': (19.3704, -99.1592),
    'EL ROSARIO': (19.5046, -99.2001),
    'ERMITA': (19.3621, -99.1428),
    'ESCUADRON 201': (19.3650, -99.1132),
    'ETIOPIA': (19.4005, -99.1565),
    'EUGENIA': (19.3853, -99.1574),
    'FERRERIA': (19.4900, -99.1728),
    'FRAY SERVANDO': (19.4216, -99.1226),
    'GARIBALDI/LAGUNILLA': (19.4444, -99.1396),
    'GENERAL ANAYA': (19.3524, -99.1457),
    'GOMEZ FARIAS': (19.4162, -99.0906),
    'GUELATAO': (19.3804, -99.0436),
    'GUERRERO': (19.4452, -99.1462),
    'HANGARES': (19.4243, -99.0880),
    'HIDALGO': (19.4375, -99.1472),
    'HOSPITAL 20 DE NOVIEMBRE': (19.3734, -99.1726),
    'HOSPITAL GENERAL': (19.4132, -99.1539),
    'IMPULSORA': (19.4975, -99.0453),
    'INDIOS VERDES': (19.4965, -99.1198),
    'INSTITUTO DEL PETROLEO': (19.4907, -99.1470),
    'INSURGENTES': (19.4237, -99.1633),
    'INSURGENTES SUR': (19.3758, -99.1783),
    'ISABEL LA CATOLICA': (19.4262, -99.1378),
    'IZTACALCO': (19.3891, -99.1091),
    'IZTAPALAPA': (19.3572, -99.0945),
    'JAMAICA': (19.4093, -99.1221),
    'JUANACATLAN': (19.4131, -99.1821),
    'JUAREZ': (19.4297, -99.1501),
    'LA PAZ': (19.3503, -98.9760),
    'LA RAZA': (19.4704, -99.1367),
    'LA VIGA': (19.4069, -99.1258),
    'LA VILLA': (19.4816, -99.1166),
    'LAGUNILLA': (19.4428, -99.1319),
    'LAZARO CARDENAS': (19.4065, -99.1432),
    'LINDAVISTA': (19.4867, -99.1373),
    'LOMAS ESTRELLA': (19.3364, -99.1065),
    'LOS REYES': (19.3582, -98.9904),
    'MARTIN CARRERA': (19.4862, -99.1044),
    'MERCED': (19.4255, -99.1246),
    'MEXICALTZINGO': (19.3582, -99.1306),
    'MIGUEL ANGEL DE QUEVEDO': (19.3414, -99.1754),
    'MISTERIOS': (19.4623, -99.1314),
    'MIXIUHCA': (19.4086, -99.1049),
    'MIXCOAC': (19.3762, -99.1873),
    'MOCTEZUMA': (19.4274, -99.1096),
    'MORELOS': (19.4390, -99.1194),
    'MUZQUIZ': (19.5015, -99.0416),
    'NATIVITAS': (19.3792, -99.1402),
    'NEZAHUALCOYOTL': (19.4933, -99.0526),
    'NINOS HEROES': (19.4191, -99.1506),
    'NOPALERA': (19.3038, -99.0494),
    'NORMAL': (19.4442, -99.1672),
    'NORTE 45': (19.4828, -99.1611),
    'OBRERA': (19.4143, -99.1438),
    'OBSERVATORIO': (19.3982, -99.2004),
    'OCEANIA': (19.4454, -99.0874),
    'OLIMPICA': (19.5212, -99.0326),
    'OLIVOS': (19.3117, -99.0620),
    'PANTEONES': (19.4526, -99.2033),
    'PANTITLAN': (19.4161, -99.0737),
    'PARQUE DE LOS VENADOS': (19.3713, -99.1590),
    'PATRIOTISMO': (19.4056, -99.1793),
    'PENON VIEJO': (19.3725, -99.0256),
    'PERIFERICO ORIENTE': (19.3248, -99.0886),
    'PINO SUAREZ': (19.4259, -99.1328),
    'PLAZA ARAGON': (19.5278, -99.0300),
    'POLANCO': (19.4336, -99.1906),
    'POLITECNICO': (19.5009, -99.1492),
    'POPOTLA': (19.4520, -99.1747),
    'PORTALES': (19.3695, -99.1425),
    'POTRERO': (19.4746, -99.1323),
    'PUEBLA': (19.4072, -99.0820),
    'REFINERIA': (19.4697, -99.1902),
    'REVOLUCION': (19.4363, -99.1546),
    'RICARDO FLORES MAGON': (19.4362, -99.1121),
    'RIO DE LOS REMEDIOS': (19.4907, -99.0469),
    'ROMERO RUBIO': (19.4409, -99.0945),
    'SALTO DEL AGUA': (19.4266, -99.1423),
    'SAN ANDRES TOMATLAN': (19.3409, -99.1158),
    'SAN ANTONIO': (19.3846, -99.1866),
    'SAN ANTONIO ABAD': (19.4187, -99.1340),
    'SAN COSME': (19.4418, -99.1610),
    'SAN JOAQUIN': (19.4452, -99.1895),
    'SAN JUAN DE LETRAN': (19.4323, -99.1408),
    'SAN LAZARO': (19.4305, -99.1147),
    'SAN PEDRO DE LOS PINOS': (19.3907, -99.1843),
    'SANTA ANITA': (19.4036, -99.1207),
    'SANTA MARTA': (19.3602, -99.0090),
    'SEVILLA': (19.4219, -99.1706),
    'TACUBA': (19.4593, -99.1887),
    'TACUBAYA': (19.4032, -99.1871),
    'TALISMAN': (19.4678, -99.1126),
    'TASQUENA': (19.3440, -99.1406),
    'TEPALCATES': (19.3912, -99.0462),
    'TEPITO': (19.4431, -99.1246),
    'TERMINAL AEREA': (19.4328, -99.0864),
    'TEZONCO': (19.3175, -99.0768),
    'TEZOZOMOC': (19.4862, -99.1934),
    'TLAHUAC': (19.2868, -99.0143),
    'TLALTENCO': (19.2925, -99.0227),
    'TLATELOLCO': (19.4542, -99.1425),
    'UAM-AZCAPOTZALCO': (19.4912, -99.1769),
    'UAM-I': (19.3512, -99.0706),
    'UNIVERSIDAD': (19.3242, -99.1736),
    'VALLE GOMEZ': (19.4589, -99.1189),
    'VALLEJO': (19.4813, -99.1534),
    'VELODROMO': (19.4087, -99.1026),
    'VIADUCTO': (19.4009, -99.1367),
    'VILLA DE ARAGON': (19.4627, -99.0601),
    'VILLA DE CORTES': (19.3879, -99.1384),
    'VIVEROS': (19.3533, -99.1755),
    'XOLA': (19.3975, -99.1378),
    'ZAPATA': (19.3707, -99.1652),
    'ZAPOTITLAN': (19.2957, -99.0322),
    'ZARAGOZA': (19.4121, -99.0824),
    'ZOCALO': (19.4328, -99.1332)
}

# 2. Conversión a DataFrame
coords_df = pd.DataFrame.from_dict(coords_metro, orient='index', columns=['LATITUD', 'LONGITUD'])
coords_df.index.name = 'ESTACION_NORMALIZADA'
coords_df.reset_index(inplace=True)

# 3. Fusión con el Dataset Principal
# Se usa LEFT JOIN para conservar todos los datos de afluencia
print(f"Fusionando coordenadas para {len(df)} registros...")
df_final = pd.merge(df, coords_df, on='ESTACION_NORMALIZADA', how='left')

# 4. Verificación de Integridad
sin_coords = df_final[df_final['LATITUD'].isna()]
if not sin_coords.empty:
    print(f"ALERTA: {len(sin_coords)} registros sin coordenadas.")
    print("Estaciones faltantes:", sin_coords['ESTACION_NORMALIZADA'].unique())
else:
    print("ÉXITO: 100% de cobertura geográfica lograda.")

Fusionando coordenadas para 1121640 registros...
ÉXITO: 100% de cobertura geográfica lograda.


In [6]:
# Filtrar filas donde LATITUD es nula (NaN)
filas_sin_coords = df_final[df_final['LATITUD'].isna()]

# Extraer los nombres únicos de las estaciones afectadas
estaciones_null = filas_sin_coords['ESTACION_NORMALIZADA'].unique()

print("-" * 40)
print(f"REPORTE DE COORDENADAS FALTANTES")
print("-" * 40)
print(f"Total de registros (filas) afectados: {len(filas_sin_coords)}")
print(f"Cantidad de estaciones únicas NULL: {len(estaciones_null)}")

if len(estaciones_null) > 0:
    print("\nListado de estaciones sin ubicar:")
    for est in estaciones_null:
        print(f" - {est}")
else:
    print("\nÉXITO: Todas las estaciones tienen latitud y longitud asignadas.")

----------------------------------------
REPORTE DE COORDENADAS FALTANTES
----------------------------------------
Total de registros (filas) afectados: 0
Cantidad de estaciones únicas NULL: 0

ÉXITO: Todas las estaciones tienen latitud y longitud asignadas.


In [8]:
# 5. Guardado Final
ruta_salida = r"Data_Tratado\Afluencia_Metro.csv"
df_final.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
print(f"Archivo generado: {ruta_salida}")

Archivo generado: Data_Tratado\Afluencia_Metro.csv
